# Sqlalchemy
http://docs.sqlalchemy.org/en/latest/core/connections.html

In [ ]:
# !pip install sqlalchemy

In [ ]:
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

### Get credentials

In [ ]:
with open('../_credentials/mysql_credentials.txt') as f:
    contents = f.read().split('\n')
    USER = contents[0]
    NAME = contents[1]
    HOST = contents[2]
    PWD = contents[3]

### Create engine, open and close engine

In [ ]:
DB = "schooldb"
credx = "mysql+pymysql://{user}:{pw}@{host}/{db}?charset=utf8".format(user=USER, host=HOST, pw=PWD, db=DB)

In [ ]:
engine = create_engine(credx, echo=False)

### Open and close connection

In [ ]:
con = engine.connect()
# do something then close
con.close()

In [ ]:
engine = create_engine(credx, echo=False)
con = engine.connect()

### Get table names

In [ ]:
engine.table_names()

In [ ]:
tables = con.execute('SHOW TABLES')
tables.fetchall()

### SQL to dataframe

In [ ]:
result = con.execute("SELECT * FROM Course")
con.close()

In [ ]:
df = pd.DataFrame(result.fetchall())
df.columns = result.keys()
df

### Open connection in context

In [ ]:
with engine.connect() as con:
    result = con.execute("SELECT * FROM Course")
df = pd.DataFrame(result.fetchall())
df.columns = result.keys()
df

In [ ]:
con = engine.connect()
df = pd.read_sql('SELECT * FROM Student;', con)
df

### SELECT

In [ ]:
query = """
SELECT course.name FROM student 
INNER JOIN enrolls_in ON student.ssn = enrolls_in.ssn 
INNER JOIN course ON course.number = enrolls_in.class
WHERE f_name = "JOHN";
"""
result = con.execute(query)

In [ ]:
query = """SELECT * FROM student"""
result = con.execute(query)
result

In [ ]:
for row in result:
    print(row)

### INSERT

In [ ]:
key = '113'
query = "INSERT INTO student VALUES ({}, 'Frank', 'Ebbers', '11111', 'Amsterdam', '94110')".format(key)

In [ ]:
con.execute(query)  # autocommits

## Abstract query to DataFrame

In [ ]:
def query_df(query, engine=engine):
    return pd.DataFrame([row for row in con.execute(query)])

In [ ]:
query_df('SELECT * FROM student ')

In [ ]:
pd.read_sql("SELECT * FROM student;", engine)

In [ ]:
pd.read_sql_table('Student', engine) # Caveat sensitive to CAPS!

In [ ]:
pd.read_sql_query("SELECT * FROM student;", con)

### Save and read dataframe as table in open database

In [ ]:
df = pd.DataFrame(np.arange(10).reshape(2,5))
df

### Write new table to database
**`if_exists`** {‘fail’, ‘replace’, ‘append’}, default ‘fail’

In [ ]:
df.to_sql('df_table', con=engine, if_exists='replace')

In [ ]:
pd.read_sql('df_table', engine)